In [26]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform # distance.euclidean(a,b)
import tensorflow as tf

In [27]:
def load_data():
    df = pd.read_csv('data/daejeon.csv', delimiter='\t', index_col=False)
    return df

In [28]:
def initialize(N, Z, I):
    '''
    Initialize theta and phi
    theta shape: N * Z
    phi shape: Z * I
    '''
    theta = np.random.rand(N, Z)
    row_sum = theta.sum(axis=1).reshape(-1, 1)
    theta = theta / row_sum
    phi = np.random.rand(Z, I)
    
    return [theta, phi]

In [29]:
def getDist(beta, location):
    '''
    input
    beta: scalar
    location: should be sorted list
    
    output
    distance: dataframe, shape of I * I
    '''
    I = len(location)
    L = np.array([[x[1], x[2]] for x in location])
    dist = squareform(pdist(np.exp(-0.5*beta*L)))
    loc_id = np.array([x[0] for x in location])
    distance = pd.DataFrame(dist, columns=loc_id, index=loc_id)
    distance[distance == 0] = 1
    return distance

In [30]:
def get_Xum(df):
    df_temp = df.sort_values('Member ID')
    df_user = df_temp[["Member ID", "Restaurant ID"]]

    x_um = {}
    for index, row in df_user.iterrows():
        if row["Member ID"] not in x_um:
            x_um[row["Member ID"]] = [row["Restaurant ID"]]
        else:
            x_um[row["Member ID"]].append(row["Restaurant ID"])

    return x_um

In [31]:
def get_P_Xum(location, df_dist, x_um, psi):
    
    loc_id = np.array([x[0] for x in location])
    phi = psi[1]; phi = np.exp(phi)
    phi = pd.DataFrame(phi, columns=loc_id)
    pXum = {}; I = len(loc_id); Z = phi.shape[0]
    for key in x_um.keys():
        temp = np.full([Z, I], np.nan)
        df_temp = pd.DataFrame(temp, columns=loc_id)
        usr_vsted_loc = x_um[key]
        userdist = df_dist.loc[usr_vsted_loc, usr_vsted_loc]
        userdistSum = userdist.sum(axis=1)
        usr_phi = phi.loc[:, usr_vsted_loc]
        prob = usr_phi * userdistSum
        prob = prob / prob.sum()

        df_temp[usr_vsted_loc] = prob
        pXum[key] = df_temp.fillna(0)
        
    return pXum

In [32]:
def E(psi, pXum, df_user, x_um):
    np.seterr(divide='ignore', invalid='ignore')
    theta = psi[0]; phi = psi[1]; topicProb = {}
    memId = sorted(df_user['Member ID'].unique())
    locId = sorted(df_user['Restaurant ID'].unique())
    
    theta = pd.DataFrame(theta, index=memId)
    for key in x_um.keys():
        theta_usr = theta.loc[key].as_matrix()
        pXum_usr = pXum[key].as_matrix()
        theta_usr = theta_usr.reshape(1, -1)
        temp = theta_usr.T * pXum_usr
        tempSum = temp.sum(axis=0)
        prob = temp / tempSum
        topicProb[key] = pd.DataFrame(prob, columns=locId).fillna(0)
            
    return topicProb

In [33]:
def get_ind(x_um, locId):
    indices = []
    memIdx = -1
    for key in x_um.keys():
        memIdx += 1
        locs = x_um[key]
        for loc in locs:
            temp_loc = locId.index(loc)
            indices.append([memIdx, temp_loc])
    
    return indices


def theta_optimize(pXum):
    theta_hat_numer = pXum.sum(axis=1)
    theta_hat_denom = theta_hat_numer.sum()
    theta_hat = theta_hat_numer / theta_hat_denom

    return theta_hat


# def fun1(phi, theta, P_hat, distance):
#     pdb.set_trace()
#     feed_dict = {P_hat : P_hat,
#                  theta : theta,
#                  phi : phi,
#                  distance : distance}
#     Q_ = sess.run(Q, feed_dict)
#     Phi_grad_ = sess.run(Phi_grad, feed_dict)
#     Phi_grad_neg = Phi_grad_.flatten()
#     negative_Phi = [-x for x in Phi_grad_neg] 
#     return -Q_, negative_Phi

def M(x_um, p_hat, Psi, distance, N, Z ,I, locId):
    theta = []
    for key in p_hat.keys():
        temp = theta_optimize(p_hat[key])
        theta.append(temp)
        
    theta = np.array(theta)
    phi = Psi[1]

    phat = []
#     PXum = []
    for key in p_hat.keys():
        phat.append(p_hat[key].as_matrix())
#         PXum.append(pXum[key].as_matrix())
    
    phat = np.array(phat)
    phat1 = np.swapaxes(phat, 0, 1)
#     PXum = np.array(PXum)
#     PXum1 = np.swapaxes(PXum, 0, 1)
    
    #N = theta.shape[0]; Z = theta.shape[1]; I = phi.shape[1]

    indices = get_ind(x_um, locId)
    Indices = tf.SparseTensor(indices = indices, values = tf.ones(len(indices), dtype = tf.float64), dense_shape = [N, I])
    
    # placeholder
    P_hat = tf.placeholder(tf.float64, shape = [Z, N, I])
    Theta = tf.placeholder(tf.float64, shape = [N, Z])
    Phi = tf.placeholder(tf.float64, shape = [Z, I])
    Dist = tf.placeholder(tf.float64, shape = [I, I])
    # Calculate P(x_um|z, R_u, Psi)
    # Z * I
    front = tf.exp(Phi)

    # N x I
    back = tf.sparse_tensor_dense_matmul(Indices, Dist)
    
    P_numer = tf.expand_dims(front, axis =1) * back
    P_denom = tf.expand_dims(tf.reduce_sum(P_numer, axis = 2), axis = 2)
    P = P_numer / P_denom

    log_Theta = tf.expand_dims(tf.transpose(tf.log(Theta)), axis = 2)
    #print("log Theta shape:", log_Theta.get_shape())
    #print("P_hat shape:", P_hat.get_shape())
    #print("P shape:", P.get_shape())

    loglike = P_hat * log_Theta * P

    Q = tf.reduce_sum(tf.sparse_tensor_dense_matmul(Indices, tf.transpose(tf.reshape(loglike, [-1, I]))))

    Phi_grad = tf.gradients(Q, Phi)
    
    feed_dict = {P_hat : phat1,
                 Theta : theta,
                 Phi : phi,
                 Dist : distance}
    
    
    sess = tf.Session()
#     pdb.set_trace()    
#     print("Check the inputs")
#     print("=============== P_hat ===============")
#     print(sess.run(P_hat, feed_dict))
#     print("=============== Theta ===============")
#     print(sess.run(Theta, feed_dict))
#     print("=============== Phi ===============")
#     print(sess.run(Phi, feed_dict))
#     print("=============== P ===============")
#     print(sess.run(P, feed_dict))
#     print("I hope they are right. God plz save us")

    Q_ = sess.run(Q, feed_dict)
    
#     print(sess.run(P_hat,feed_dict))
#     print("-------------------------")
#     print(sess.run(Theta, feed_dict))
#     print("-------------------------")
#     print(sess.run(Phi))
#     print("-------------------------")
#     print(sess.run(P))

    Phi_grad_ = sess.run(Phi_grad, feed_dict)
    print("phi:", type(phi))
    print("theta:", type(theta))
    print("phat1:", type(phat1))
    print("distance:", type(distance))
    fun = lambda phi : fun1(phi, theta, phat1, distance)
    
    res =  minimize(fun, phi, jac = True)
    
    return [theta, res.x]


In [52]:
def M(x_um, p_hat, Psi, distance, N, Z ,I, locId):
    theta = []
    for key in p_hat.keys():
        temp = theta_optimize(p_hat[key])
        theta.append(temp)
        
    theta = np.array(theta)
    phi = Psi[1]

    phat = []
#     PXum = []
    for key in p_hat.keys():
        phat.append(p_hat[key].as_matrix())
#         PXum.append(pXum[key].as_matrix())
    
    phat = np.array(phat)
    phat1 = np.swapaxes(phat, 0, 1)
#     PXum = np.array(PXum)
#     PXum1 = np.swapaxes(PXum, 0, 1)
    
    #N = theta.shape[0]; Z = theta.shape[1]; I = phi.shape[1]

    indices = get_ind(x_um, locId)
    Indices = tf.SparseTensor(indices = indices, values = tf.ones(len(indices), dtype = tf.float64), dense_shape = [N, I])
    
    # placeholder
    P_hat = tf.placeholder(tf.float64, shape = [Z, N, I])
    Theta = tf.placeholder(tf.float64, shape = [N, Z])
    Phi = tf.placeholder(tf.float64, shape = [Z, I])
    Dist = tf.placeholder(tf.float64, shape = [I, I])
    # Calculate P(x_um|z, R_u, Psi)
    # Z * I
    front = tf.exp(Phi)

    # N x I
    back = tf.sparse_tensor_dense_matmul(Indices, Dist)
    
    P_numer = tf.expand_dims(front, axis =1) * back
    P_denom = tf.expand_dims(tf.reduce_sum(P_numer, axis = 2), axis = 2)
    P = P_numer / P_denom

    log_Theta = tf.expand_dims(tf.transpose(tf.log(Theta)), axis = 2)
    #print("log Theta shape:", log_Theta.get_shape())
    #print("P_hat shape:", P_hat.get_shape())
    #print("P shape:", P.get_shape())

    loglike = P_hat * log_Theta * P

    Q = -tf.reduce_sum(tf.sparse_tensor_dense_matmul(Indices, tf.transpose(tf.reshape(loglike, [-1, I]))))
    Phi_grad = tf.negative(tf.gradients(Q, Phi))
    
    sess = tf.Session()
#     pdb.set_trace()
    def objective(phi_):
        phi_ = phi_.reshape(Z, I)
        #phat_, theta_, phi_, distance_ = param
        feed_dict={P_hat: phat1, Theta: theta, Phi: phi_, Dist: distance}
        
        print("=========================== P_hat (Objective) ================================")
        print(sess.run(P_hat, feed_dict))
        print("p_hat:", P_hat.shape)
        print("=========================== Theta (Objective) ================================")
        print(sess.run(Theta, feed_dict))
        print("Theta:", Theta.shape)
        print("=========================== Phi (Objective) ================================")
        print(sess.run(Phi, feed_dict))
        print("Phi:", Phi.shape)
        print("=========================== Dist (Objective) ================================")
        print(sess.run(Dist, feed_dict))
        print("Dist:", Dist.shape)
        return sess.run(Q, feed_dict={P_hat: phat1, Theta: theta, Phi: phi_, Dist: distance})
    
    def gradient(phi_):
        phi_ = phi_.reshape(Z, I)
        #phat_, theta_, phi_, distance_ = param
        feed_dict={P_hat: phat1, Theta: theta, Phi: phi_, Dist: distance}
        ret = sess.run(Phi_grad, feed_dict={P_hat: phat1, Theta: theta, Phi: phi_, Dist: distance})
        print("=========================== P_hat (gradient) ================================")
        print(sess.run(P_hat, feed_dict))
        print("p_hat:", P_hat.shape)
        print("=========================== Theta (gradient) ================================")
        print(sess.run(Theta, feed_dict))
        print("Theta:", Theta.shape)
        print("=========================== Phi (gradient) ================================")
        print(sess.run(Phi, feed_dict))
        print("Phi:", Phi.shape)
        print("=========================== Dist (gradient) ================================")
        print(sess.run(Dist, feed_dict))
        print("Dist:", Dist.shape)
        res = np.squeeze(ret).flatten()
        return res
                       
    res =  minimize(objective, x0=phi, jac=gradient)
    
    return [theta, res.x]


In [53]:
def main():
    df = load_data()
    beta = float(input("Enter the beta value:"))
    Z = int(input("Enter the number of topic:"))
    N = len(df['Member ID'].unique())
    I = len(df['Restaurant ID'].unique())

    df_loc = df[['Restaurant ID', 'Restaurant Latitude', 'Restaurant Longitude']]
    df_user = df[['Member ID', 'Restaurant ID']]
    location = sorted(list(set([tuple(x) for x in df_loc.to_records(index=False)])))

    locId = sorted(df_user['Restaurant ID'].unique())
    memId = sorted(df_user['Member ID'].unique())
    df_dist = getDist(beta, location)

    x_um = get_Xum(df)
    
    Psi = initialize(N, Z, I)
    Theta = Psi[0]; Phi = Psi[1]


    pXum = get_P_Xum(location, df_dist, x_um, Psi)    
    #pdb.set_trace()
    
    
    P_hat = E(Psi, pXum, df_user, x_um)

#     p_hat = []
#     PXum = []
#     for key in P_hat.keys():
#         p_hat.append(P_hat[key].as_matrix())
#         PXum.append(pXum[key].as_matrix())
    
#     p_hat = np.array(p_hat)
#     p_hat1 = np.swapaxes(p_hat, 0, 1)
#     PXum = np.array(PXum)
#     PXum1 = np.swapaxes(PXum, 0, 1)
    
#     theta = []
#     for key in P_hat.keys():
#         temp = theta_optimize(P_hat[key])
#         theta.append(temp)
        
#     theta = np.array(theta)
    # P :  pXum
    distance = df_dist.as_matrix()
    psi = M(x_um, P_hat, Psi, distance, N, Z, I, locId)
    
    return psi

In [54]:
res = main()

Enter the beta value:1
Enter the number of topic:5
=========================== P_hat (gradient) ================================
[[[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.

In [56]:
# print(res)
theta = res[0]; phi = res[1].reshape(5, 852)
print(theta.shape)
print(phi.shape)
print(theta)

(1153, 5)
(5, 852)
[[ 0.14130354  0.00667859  0.3299345   0.22163854  0.30044484]
 [ 0.33937857  0.13019507  0.02758359  0.13472319  0.36811957]
 [ 0.05594476  0.19283773  0.26268841  0.34703787  0.14149123]
 ..., 
 [ 0.23887982  0.19820294  0.3245353   0.10394149  0.13444046]
 [ 0.09074303  0.17800091  0.39590659  0.2227503   0.11259916]
 [ 0.22629793  0.17299301  0.15714935  0.19703903  0.24652069]]


In [57]:
def getDist(beta, location):
    '''
    input
    beta: scalar
    location: should be sorted list
    
    output
    distance: dataframe, shape of I * I
    '''
    I = len(location)
    L = np.array([[x[1], x[2]] for x in location])
    dist = squareform(pdist(np.exp(-0.5*beta*L)))
    loc_id = np.array([x[0] for x in location])
    distance = pd.DataFrame(dist, columns=loc_id, index=loc_id)
    distance[distance == 0] = 1
    return distance

In [125]:
df = load_data()
beta = 1; Z = 5
df_loc = df[['Restaurant ID', 'Restaurant Latitude', 'Restaurant Longitude']]
df_user = df[['Member ID', 'Restaurant ID']]
location = sorted(list(set([tuple(x) for x in df_loc.to_records(index=False)])))
locId = sorted(df_user['Restaurant ID'].unique()); memId = sorted(df_user['Member ID'].unique())
I = len(locId); N = len(memId)
Psi = initialize(N, Z ,I); theta = Psi[0]; phi = Psi[1]

# current: 대전광역시 서구, 탄방동
c = [36.344069, 127.398756]


L = np.array([[x[1], x[2]] for x in location])
distCur = []
for loc in L:
    temp = np.exp(-0.5 * beta * np.linalg.norm(loc-c))
    distCur.append(temp)
    
distCur = np.array(distCur).reshape(1, -1)
cur_numer = phi * distCur
cur_denom = cur_numer.sum(axis=1).reshape(-1, 1)
cur = cur_numer / cur_denom
prob = theta @ cur

# test_prob = list(prob[0])
# print(test_prob.index(max(test_prob)))
# print(prob[0][631])
# print(locId[631])
# test = df[df['Restaurant ID'] == locId[631]]
# # print(df['Restaurant Name'].where(df['Restaurant ID'] == locId[631]))
# print(test)
# print(test['Restaurant Name'].unique())

# best_prob = prob.max(axis=1)
# print(best_prob)

best_locId = np.argmax(prob, axis=1)
best_loc = [locId[x] for x in best_locId]
recommendation = []
for loc in best_loc:
    rest = df[df['Restaurant ID']==loc]['Restaurant Name'].unique()
    recommendation.append(rest[-1])
    
print(recommendation)


def test(L, c, Psi):
    theta = Psi[0]; phi = Psi[1]
    distCur = []
    for loc in L:
        temp = np.exp(-0.5 * beta * np.linalg.norm(loc-c))
        distCur.append(temp)
        
    distCur = np.array(distCur).reshape(1, -1)
    cur_numer = phi * distCur
    cur_denom = cur_numer.sum(axis=1).reshape(-1, 1)
    cur = cur_numer / cur_denom
    res = theta @ cur
    
    return res

def find_recommendation(user_prob, locId, df):
    best_locId = np.argmax(prob, axis=1)
    best_loc = [locId[x] for x in best_locId]
    recommendation = []
    for loc in best_loc:
        rest = df[df['Restaurant ID']==loc]['Restaurant Name'].unique()
        recommendation.append(rest[-1])
        
    return recommendation

['셀로스터즈', '악어식탁', '비단강숯불장어', '아저씨돈까스', '꽃', '셀로스터즈', '고베규카츠', '셀로스터즈', '고베규카츠', '악어식탁', '유달리', '미랑참치', '아저씨돈까스', '셀로스터즈', '아저씨돈까스', '악어식탁', '비단강숯불장어', '악어식탁', '아저씨돈까스', '악어식탁', '라라코스트', '미랑참치', '아저씨돈까스', '아저씨돈까스', '다솜차반', '미랑참치', '미랑참치', '병규돈가스', '아저씨돈까스', '악어식탁', '비단강숯불장어', '썸띵라이크', '유달리', '아저씨돈까스', '악어식탁', '비단강숯불장어', '비단강숯불장어', '비단강숯불장어', '악어식탁', '병규돈가스', '아저씨돈까스', '고베규카츠', '악어식탁', '병규돈가스', '악어식탁', '셀로스터즈', '비단강숯불장어', '병규돈가스', '악어식탁', '셀로스터즈', '아저씨돈까스', '아저씨돈까스', '악어식탁', '아저씨돈까스', '악어식탁', '비단강숯불장어', '다솜차반', '고베규카츠', '미랑참치', '병규돈가스', '비단강숯불장어', '비단강숯불장어', '악어식탁', '다솜차반', '악어식탁', '유달리', '병규돈가스', '악어식탁', '비단강숯불장어', '악어식탁', '셀로스터즈', '아저씨돈까스', '다솜차반', '비단강숯불장어', '비단강숯불장어', '아저씨돈까스', '셀로스터즈', '미랑참치', '고베규카츠', '악어식탁', '다솜차반', '병규돈가스', '다솜차반', '악어식탁', '비단강숯불장어', '병규돈가스', '악어식탁', '비단강숯불장어', '셀로스터즈', '비단강숯불장어', '아저씨돈까스', '악어식탁', '로보쿡', '악어식탁', '악어식탁', '로보쿡', '비단강숯불장어', '꽃', '셀로스터즈', '썸띵라이크', '병규돈가스', '셀로스터즈', '아저씨돈까스', '악어식탁', '아저씨돈까스', '썸띵라이크', '셀로스터즈', '셀로스터즈', '셀로스터즈', '병규돈가스', '악어식탁', '미랑참

In [74]:
a = np.random.rand(2,4)
print(a)
b = np.random.rand(1,4)
print(b)
c = a * b
print(c.shape)

[[ 0.72161465  0.07087316  0.15959468  0.84321623]
 [ 0.23567683  0.63640637  0.42515338  0.11255727]]
[[ 0.6347933   0.43205474  0.27790426  0.76315736]]
(2, 4)
